In [11]:
import serial
import time
import re
import csv
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from os.path import exists

from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains



product_dict = {'B00TX95XRE': 
                {'URL': 'https://www.amazon.in/dp/B00TX95XRE',
                 'Name': 'Parrys White Label Sugar',
                 'Quantity': 100},
                
                 'B074N7VHV4':
                {'URL': 'https://www.amazon.in/dp/B074N7VHV4',
                 'Name': 'Tata Sampann Unpolished Toor Dal',
                 'Quantity': 150}
               }

account_email_id = 'suryavp9919@gmail.com'
account_password = 'Surya@558'



def checkout_product(product_ID):
    delay_in_seconds = 3

    amazon_main_URL = 'https://www.amazon.in/'
    
    current_product_URL = product_dict[product_ID]['URL']

    browser = Firefox()
    browser.maximize_window()
    browser.get(amazon_main_URL)
    
    # To wait for the results to load
    time.sleep(delay_in_seconds)

    signInBtn = browser.find_element(By.ID, 'nav-link-accountList')
    signInBtn.click()

    
    time.sleep(delay_in_seconds)

    emailTxtBox = browser.find_element(By.ID, 'ap_email')
    emailTxtBox.send_keys(account_email_id)
    emailTxtBox.submit()


    time.sleep(delay_in_seconds)

    passwordTxtBox = browser.find_element(By.ID, 'ap_password')
    passwordTxtBox.send_keys(account_password)
    passwordTxtBox.submit()


    time.sleep(delay_in_seconds)

    browser.get(current_product_URL)
    
    
    time.sleep(delay_in_seconds)

    addToCartBtn = browser.find_element(By.ID, 'add-to-cart-button')
    addToCartBtn.click()
    
    time.sleep(delay_in_seconds)
    
    
    try:
        closeSidebarBtn = browser.find_element(By.ID, 'sis-close-div')
        closeSidebarBtn.click()
    
        time.sleep(delay_in_seconds)
    except:
        print('Sidebar not found')
    
    
    cartBtn = browser.find_element(By.ID, 'nav-cart')
    cartBtn.click()
    
    
    time.sleep(delay_in_seconds)
    
    
    buyNowBtn = browser.find_element(By.ID, 'desktop-ptc-button-celWidget')
    buyNowBtn.click()
    
    time.sleep(delay_in_seconds)
    
    try:
        continueBtn = browser.find_element(By.ID, 'orderSummaryPrimaryActionBtn')
        continueBtn.click()
        time.sleep(delay_in_seconds)
    except:
        print('Continue button wasn\'t present')
    
    
    #cashOnDeliveryBtn = browser.find_element(By.CSS_SELECTOR, 'span[Cash On Delivery/Pay On Delivery]')
    #cashOnDeliveryBtn.click()
        
    #time.sleep(delay_in_seconds)
    
    
    #useThisPaymentBtn = browser.find_element(By.ID, 'orderSummaryPrimaryActionBtn-announce')
    #useThisPaymentBtn.click()
    
    #time.sleep(delay_in_seconds)
    
    #placeOrderBtn = browser.find_element(By.ID, 'submitOrderButtonId-announce')
    #placeOrderBtn.click()
    
    
    quantity_of_product = product_dict[product_ID]['Quantity']
    return quantity_of_product
    
    

if __name__ == '__main__':
   
    try:
        ser = serial.Serial('COM7', 57600)
        print('Serial connection established')
    except:
        print("Serial connection already exists")
        exit()
        
    data_dir = 'D:\Final Year Project\Project'
    path_to_readings_file = data_dir + 'quantity.csv'
    
    # Set which product is being measured
    product_ID = 'B00TX95XRE'
    checkout_product(product_ID)
    
    product_quantity = product_dict[product_ID]['Quantity']
    current_value = 0
    previous_value = 0
    Quantity=0 
    
    # Check if csv exists
    file_exists = exists(path_to_readings_file)
    if file_exists:
        with open(path_to_readings_file, mode ='r') as file:
            # Reading the CSV file
            csvFile = csv.DictReader(file)
                        
            for line in csvFile:
                
                Quantity = float(line['quantity_left'])
                current_value = float(line['consumption'])
            print('CSV data loaded\n')

    counter = 0
    measure_interval = 50
    data_list = list()
    threshold = 20
       
    # plastic container without lid
    container_weight = 0
    
    
    while True:
        #time.sleep(1)
        
        # Read value
        str_rn = ser.readline().decode().strip()
        # print(f"from ardrino {str_rn} ")
        # Skip process if string detected
        has_text = re.search('[a-zA-Z]', str_rn)
        if has_text or not str_rn:
            continue
            
        data_list.append(float(str_rn))
        counter += 1
        
        if counter >= measure_interval:
            previous_value = current_value
            
            readings_to_pick = 20 
            picked_readings = sorted(data_list, reverse = True)[:readings_to_pick]
            average_reading = sum(picked_readings) / readings_to_pick
            
            current_value = int(average_reading - container_weight)
            
            date_obj = datetime.now()
            print(date_obj.strftime("%d/%m/%Y %H:%M:%S"), ' - ', current_value, sep='')
            
            counter = 0
            data_list = list()
        else:
            continue
        
        # Container left empty
        if current_value in range(container_weight - 2, container_weight + 2):
            print('Container empty')
            None
            
        # Container not returned to position
        elif current_value in range(-container_weight - 2, 1):
            # Container not returned to position
            print('Container was not placed back on the scale')
            None
                
        # No changes in value
        elif (previous_value - current_value) in range(-2, 3):
            print('No changes in value was found')
            None
                
        # Container recently filled up
        elif previous_value < current_value:
            Quantity =current_value
            print('Container was recently filled up')
            
                
        # Normal usage
        elif current_value < previous_value:
            consumption = previous_value - current_value
            
            
            print('Consumption -', consumption)
            
            # Record entry
            
            date_obj = datetime.now()
            
            file_exists = exists(path_to_readings_file)
            
            with open(path_to_readings_file, 'a') as csvfile:
                headers = ['date', 'time', 'quantity_left', 'consumption']
                writer = csv.DictWriter(csvfile, delimiter = ',', lineterminator = '\n', fieldnames = headers)

                # If file doesn't exist, write headers
                if not file_exists:
                    print('CSV created')
                    writer.writeheader()

                writer.writerow({headers[0]: date_obj.strftime('%d/%m/%Y'),
                                 headers[1]: date_obj.strftime('%H:%M:%S'),
                                 headers[2]: Quantity,
                                 headers[3]: consumption})
                Quantity -= consumption
            
            with open(path_to_readings_file, mode ='r') as file:
                # Reading the CSV file
                csvFile = csv.DictReader(file)

                x = list()
                y = list()

                print('--- CSV ---')
                for lines in csvFile:
                    
                    print(lines)

                    # Check if need to order
                    product_quantity_record = lines['quantity_left']
                    datetime_str = lines['date'] + ' ' + lines['time']
                    datetime_object = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M:%S')
                    unix_time = time.mktime(datetime_object.timetuple())
                    x.append(int(unix_time))
                        
                    
                    y.append(int(float(product_quantity_record)))
                print('--- CSV ---')

                    
                if len(x) <= 5 or len(y) <= 5:
                    continue
                    
                slope, intercept, r, p, std_err = stats.linregress(x, y)

                def y_predict(x):
                    return slope * x + intercept
                
                def x_predict(y):
                    return (y - intercept) / slope

                mymodel = list(map(y_predict, x))

                last_reading_time = datetime.fromtimestamp(x[-1])
                
                # Get date on when quantity reaches 20 grams
                depletion_date = datetime.fromtimestamp(x_predict(threshold))
                
                plt.scatter(x, y)
                plt.plot(x, mymodel)
                plt.plot(x[-1], threshold, color='red')
                plt.show()
                
                print('Product will be depleted on the estimated date of', depletion_date.strftime('%d/%m/%Y %H:%M:%S'))
                
                # If product will be depleted in 3 days
                if (depletion_date - last_reading_time).days < 3:
                    # Pass order
                    checkout_product(product_ID)
                    print("Order Placed and Filled up the container")
                    
                    # print('Order on', product_dict[product_ID]['Name'], 'passed')
                    
                    # # Add stock to database
                    Quantity += product_dict[product_ID]['Quantity']
                    
        print()

            
    ser.close()
    exit()

Continue button wasn't present
